In [1]:
# this is a test to see if GitHub is derping
import pandas as pd
df = pd.read_csv('heart_disease_uci(1).csv')

df.head

<bound method NDFrame.head of      age     sex        dataset               cp  trestbps   chol    fbs  \
0     63    Male      Cleveland   typical angina     145.0  233.0   True   
1     67    Male      Cleveland     asymptomatic     160.0  286.0  False   
2     67    Male      Cleveland     asymptomatic     120.0  229.0  False   
3     37    Male      Cleveland      non-anginal     130.0  250.0  False   
4     41  Female      Cleveland  atypical angina     130.0  204.0  False   
..   ...     ...            ...              ...       ...    ...    ...   
915   54  Female  VA Long Beach     asymptomatic     127.0  333.0   True   
916   62    Male  VA Long Beach   typical angina       NaN  139.0  False   
917   55    Male  VA Long Beach     asymptomatic     122.0  223.0   True   
918   58    Male  VA Long Beach     asymptomatic       NaN  385.0   True   
919   62    Male  VA Long Beach  atypical angina     120.0  254.0  False   

              restecg  thalch  exang  oldpeak        slop